In [32]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import collections
import scipy.stats
import statistics
from Bio import SeqIO
from itertools import product
from scipy.fftpack import fft, ifft
import math 

import operator
import time

In [43]:

def ExtractionTechniques_headerNAC(k, foutput):
    global listTotal, caracteres
    listTotal = []
    file = open(foutput, 'a')
    file.write('%s,' % ('nameseq'))
    permsList = [''.join(str(i) for i in x) for x in product(caracteres, repeat=k)]
    # print (permsList)
    for perm in permsList:
        # print (perm)
        listTotal.append(perm)
        file.write('%s,' % (str(perm)))
    file.write('label')
    file.write('\n')
    return


def ExtractionTechniques_chunks(seq, win, step):
    seqlen = len(seq)
    for i in range(0, seqlen, step):
        j = seqlen if i+win>seqlen else i+win
        yield seq[i:j]
        if j==seqlen: break
    return        
    

def ExtractionTechniques_chunks_two(seq, win):
    seqlen = len(seq)
    for i in range(seqlen):
        j = seqlen if i+win>seqlen else i+win
        yield seq[i:j]
        if j==seqlen: break
    return

            
def ExtractionTechniques_file_record(name_seq, foutput, labelDataset):
    file = open(foutput, 'a')
    file.write('%s,' % (name_seq))
    for x in probabilities:
        # print (x)
        file.write('%s,' % (str(x[1])))
    file.write(labelDataset)
    file.write('\n')
    print ('Recorded Sequence: %s' % (name_seq))
    return
    

# Nucleic acid composition (NAC), Di-nucleotide composition (DNC), Tri-nucleotide composition (TNC)
def nacSeq(k, foutput, finput, labelDataset):
    global probabilities
    ExtractionTechniques_headerNAC(k, foutput)
    for seq_record in SeqIO.parse(finput, 'fasta'):
        seq = seq_record.seq
        seq = seq.upper()	
        name_seq = seq_record.name
        probabilities = []
        kmer = {}
        totalWindows = (len(seq) - k) + 1 # (L - k + 1)
        permsList = [''.join(str(i) for i in x) for x in product(caracteres, repeat=k)]
        for key in permsList:
            kmer[key] = 0
        kmer = collections.OrderedDict(sorted(kmer.items()))
        for subseq in ExtractionTechniques_chunks_two(seq, k):
            if subseq in kmer:
                # print(subseq)
                kmer[subseq] = kmer[subseq] + 1
            else:
                kmer[subseq] = 1
        for key, value in kmer.items():
            # print (key)
            # print (value)
            probabilities.append([str(key), value/totalWindows])
        ExtractionTechniques_file_record(name_seq, foutput, labelDataset)
    return


def ExtractionTechniques(finput, foutput, tech):
    global listTotal, caracteres, complement
    labelDataset = 'DNA'
    # tech = str(args.type)
    seq = 1
    stepw = 1
    if seq == 1:
        caracteres = ['A', 'C', 'G', 'T']
        complement = {'A': 'T','C': 'G','G': 'C','T': 'A'}
    else:
        caracteres = ['A', 'C', 'G', 'U']
        complement = {'A': 'U','C': 'G','G': 'C','U': 'A'}
    if tech == 'NAC' or tech == 'nac':
        nacSeq(1, foutput, finput, labelDataset)
    elif tech == 'DNC' or tech == 'dnc':
        nacSeq(2, foutput, finput, labelDataset)
    elif tech == 'TNC' or tech == 'tnc':
        nacSeq(3, foutput, finput, labelDataset)
#############################################################################

In [36]:
def header(foutput):
	dataset = open(foutput, 'a')
	dataset.write("nameseq,average,median,maximum,minimum,peak,"
                  + "none_levated_peak,sample_standard_deviation,population_standard_deviation,"
                  + "percentile15,percentile25,percentile50,percentile75,amplitude,"
                  + "variance,interquartile_range,semi_interquartile_range,"
                  + "coefficient_of_variation,skewness,kurtosis,label")
	dataset.write("\n")
	return

        
def file_record(foutput, label_dataset):
    dataset = open(foutput, 'a')
    dataset.write("%s," % (str(name_seq)))
    for metric in features:
        dataset.write("%s," % (metric))
        # dataset.write("{0:.4f},".format(metric))
    dataset.write(label_dataset)
    dataset.write("\n")
    print("Sequence Analyzed!!")
    return


def feature_extraction():
    global features
    features = []
    average = sum(spectrum)/len(spectrum)
    features.append(average)
    ###################################
    median = np.median(spectrum)
    features.append(median)
	###################################
    maximum = np.max(spectrum)
    features.append(maximum)
    ###################################
    minimum = np.min(spectrum)
    features.append(minimum)
    ###################################
    peak = (len(spectrum)/3)/(average)
    features.append(peak)
    ###################################
    peak_two = (len(spectrumTwo)/3)/(np.mean(spectrumTwo))
    features.append(peak_two)
    ###################################
    standard_deviation = np.std(spectrum) # standard deviation
    features.append(standard_deviation)
    ###################################
    standard_deviation_pop = statistics.stdev(spectrum) # population sample standard deviation 
    features.append(standard_deviation_pop)
    ###################################
    percentile15 = np.percentile(spectrum, 15)
    features.append(percentile15)
    ###################################
    percentile25 = np.percentile(spectrum, 25)
    features.append(percentile25)
    ###################################
    percentile50 = np.percentile(spectrum, 50)
    features.append(percentile50)
    ###################################
    percentile75 = np.percentile(spectrum, 75)
    features.append(percentile75)
    ###################################
    amplitude = maximum - minimum
    features.append(amplitude)
    ###################################
    # mode = statistics.mode(spectrum)
    ###################################
    variance = statistics.variance(spectrum)
    features.append(variance)
    ###################################
    interquartile_range = np.percentile(spectrum, 75) - np.percentile(spectrum, 25)
    features.append(interquartile_range)
    ###################################
    semi_interquartile_range = (np.percentile(spectrum, 75) - np.percentile(spectrum, 25))/2 
    features.append(semi_interquartile_range)
    ###################################
    coefficient_of_variation = standard_deviation/average
    features.append(coefficient_of_variation)
    ###################################
    skewness = (3 * (average - median))/standard_deviation
    features.append(skewness)   
    ###################################
    kurtosis = (np.percentile(spectrum, 75) - np.percentile(spectrum, 25)) / (2 * (np.percentile(spectrum, 90) - np.percentile(spectrum, 10))) 
    features.append(kurtosis)
    ###################################
    return


def real_fourier(finput, foutput, label_dataset):
    header(foutput)
    global spectrum, spectrumTwo, name_seq
    for seq_record in SeqIO.parse(finput, "fasta"):
        seq = seq_record.seq
        seq = seq.upper()
        name_seq = seq_record.name
        spectrum = []
        spectrumTwo = []
        real = []
        for nucle in seq:
            if nucle == "T" or nucle == "U":
                real.append(1.5)
            elif nucle == "C":
                real.append(0.5)
            elif nucle == "A":
                real.append(-1.5)
            else:
                real.append(-0.5)
        FR = fft(real)
        for i in range(len(seq)):
            specTotal = (abs(FR[i])**2)
            specTwo = (abs(FR[i]))
            spectrum.append(specTotal)
            spectrumTwo.append(specTwo)
        feature_extraction()
        file_record(foutput, label_dataset)
    return


#############################################################################    
def FourierClass(finput, foutput, label_dataset):
    real_fourier(finput, foutput, label_dataset)
#############################################################################

In [37]:
def EntropyClass_header(foutput, ksize):
    file = open(foutput, 'a')
    file.write("nameseq,")
    for i in range(1, ksize+1):
        file.write("k" + str(i) + ",")
    file.write("label")
    file.write("\n")
    return


def EntropyClass_chunks_two(seq, win):
    seqlen = len(seq)
    for i in range(seqlen):
        j = seqlen if i+win>seqlen else i+win
        yield seq[i:j]
        if j==seqlen: break
    return

            
def EntropyClass_file_record(foutput, label_dataset):
    file = open(foutput, 'a')
    file.write("%s," % (name_seq))
    for data in information_entropy:
        file.write("%s," % (str(data)))
    file.write(label_dataset)
    file.write("\n")
    print ("Recorded Sequence!!!")
    return
    

def entropy_equation(ksize, finput, foutput, e, label_dataset):
    EntropyClass_header(foutput, ksize)
    global name_seq, information_entropy
    for seq_record in SeqIO.parse(finput, "fasta"):
        seq = seq_record.seq
        seq = seq.upper()
        name_seq = seq_record.name
        information_entropy = []
        for k in range(1, ksize+1):
            probabilities = []
            kmer = {}
            total_windows = (len(seq) - k) + 1 # (L - k + 1)
            for subseq in EntropyClass_chunks_two(seq, k):
                if subseq in kmer:
                    # print(subseq)
                    kmer[subseq] = kmer[subseq] + 1
                else:
                    kmer[subseq] = 1
            for key, value in kmer.items():
                # print(key)
                # print(value)
                probabilities.append(value/total_windows)
            if e == "Shannon" or e == "shannon":
                entropy_equation = [(p * math.log(p, 2)) for p in probabilities]
                entropy = -(sum(entropy_equation))
                information_entropy.append(entropy)
            else:
                q = 2
                entropy_equation = [(p ** q) for p in probabilities]
                entropy =  (1/(q - 1)) * (1 - sum(entropy_equation))
                information_entropy.append(entropy)
        EntropyClass_file_record(foutput, label_dataset)
    return

        
#############################################################################    
def EntropyClass(finput, foutput):
    ksize = 10
    stepw = 1
    e = 'Shannon'
    label_dataset = 'mRNA'
    entropy_equation(ksize, finput, foutput, e, label_dataset)
       
#############################################################################

In [45]:
def AccumulatedNucleotideFrequency_check_files(dataset_labels):
    for name, label in dataset_labels.items():
        if os.path.exists(name):
            print('Dataset %s: Found File' % (name))
            run = 1
        else:
            print('Dataset %s: File not exists' % (name))
            run = 0
            break
    return run

def AccumulatedNucleotideFrequency_header_fourier(foutput):
    dataset = open(foutput, 'a')
    dataset.write('nameseq,average,median,maximum,minimum,peak,'
                  + 'none_levated_peak,sample_standard_deviation,population_standard_deviation,'
                  + 'percentile15,percentile25,percentile50,percentile75,amplitude,'
                  + 'variance,interquartile_range,semi_interquartile_range,'
                  + 'coefficient_of_variation,skewness,kurtosis,label')
    dataset.write('\n')
    return foutput


def AccumulatedNucleotideFrequency_file_record_fourier(features, name_seq, label_dataset, foutput):
    dataset = open(foutput, 'a')
    dataset.write('%s,' % (str(name_seq)))
    for metric in features:
        dataset.write('%s,' % (metric))
        # dataset.write('{0:.4f},'.format(metric))
    dataset.write(label_dataset)
    dataset.write('\n')
    print('Recorded Sequence: %s' % (name_seq))
    return


def AccumulatedNucleotideFrequency_feature_extraction(features, spectrum, spectrumTwo):
    average = sum(spectrum)/len(spectrum)
    features.append(average)
    ###################################
    median = np.median(spectrum)
    features.append(median)
	###################################
    maximum = np.max(spectrum)
    features.append(maximum)
    ###################################
    minimum = np.min(spectrum)
    features.append(minimum)
    ###################################
    peak = (len(spectrum)/3)/(average)
    features.append(peak)
    ###################################
    peak_two = (len(spectrumTwo)/3)/(np.mean(spectrumTwo))
    features.append(peak_two)
    ###################################
    standard_deviation = np.std(spectrum) # standard deviation
    features.append(standard_deviation)
    ###################################
    standard_deviation_pop = statistics.stdev(spectrum) # population sample standard deviation 
    features.append(standard_deviation_pop)
    ###################################
    percentile15 = np.percentile(spectrum, 15)
    features.append(percentile15)
    ###################################
    percentile25 = np.percentile(spectrum, 25)
    features.append(percentile25)
    ###################################
    percentile50 = np.percentile(spectrum, 50)
    features.append(percentile50)
    ###################################
    percentile75 = np.percentile(spectrum, 75)
    features.append(percentile75)
    ###################################
    amplitude = maximum - minimum
    features.append(amplitude)
    ###################################
    # mode = statistics.mode(spectrum)
    ###################################
    variance = statistics.variance(spectrum)
    features.append(variance)
    ###################################
    interquartile_range = np.percentile(spectrum, 75) - np.percentile(spectrum, 25)
    features.append(interquartile_range)
    ###################################
    semi_interquartile_range = (np.percentile(spectrum, 75) - np.percentile(spectrum, 25))/2 
    features.append(semi_interquartile_range)
    ###################################
    coefficient_of_variation = standard_deviation/average
    features.append(coefficient_of_variation)
    ###################################
    skewness = (3 * (average - median))/standard_deviation
    features.append(skewness)   
    ###################################
    kurtosis = (np.percentile(spectrum, 75) - np.percentile(spectrum, 25)) / (2 * (np.percentile(spectrum, 90) - np.percentile(spectrum, 10))) 
    features.append(kurtosis)
    ###################################
    return


def AccumulatedNucleotideFrequency_accumulated_nucle_frequency_fourier(dataset_labels, foutput):
    AccumulatedNucleotideFrequency_header_fourier(foutput)
    for finput, label in dataset_labels.items():
        for seq_record in SeqIO.parse(finput, 'fasta'):
            seq = seq_record.seq
            seq = seq.upper()
            name_seq = seq_record.name
            features = []
            spectrum = []
            spectrumTwo = []
            mapping = []
            A = 0
            C = 0
            T = 0
            G = 0
            for i in range(len(seq)):
                if seq[i] == 'A':
                    A += 1
                    mapping.append(A / (i + 1))
                elif seq[i] == 'C':
                    C += 1
                    mapping.append(C / (i + 1))
                elif seq[i] == 'T' or seq[i] == 'U':
                    T += 1
                    mapping.append(T / (i + 1))
                else:
                    G += 1
                    mapping.append(G / (i + 1))
            Fmap = fft(mapping)
            for i in range(len(mapping)):
                specTotal = (abs(Fmap[i])**2)
                specTwo = (abs(Fmap[i]))
                spectrum.append(specTotal)
                spectrumTwo.append(specTwo)
            AccumulatedNucleotideFrequency_feature_extraction(features, spectrum, spectrumTwo)
            AccumulatedNucleotideFrequency_file_record_fourier(features, name_seq, label, foutput)
    return


#############################################################################
#############################################################################   
def AccumulatedNucleotideFrequency(name, foutput, label):
    n = 1
    dataset_labels = {}
    for i in range(1, n + 1):
        dataset_labels[name] = label
    if AccumulatedNucleotideFrequency_check_files(dataset_labels) == 1:
        AccumulatedNucleotideFrequency_accumulated_nucle_frequency_fourier(dataset_labels, foutput)


In [40]:
def des_start_code(codes):
	if codes in ('ATG', 'AUG'):
		return True
	return False


def des_end_code(codes):
	if codes in ('TAA', 'UAA', 'TAG', 'UAG', 'TGA', 'UGA'):
		return True
	return False


def read_by_three(string, offset):
	flag = True
	length = len(string)
	start = end = -1
	i = 0
	result = set()
	while i < length-2:
		codes = string[i:i+3]
		if des_start_code(codes) and flag:
			start = i
			flag = False
		if des_end_code(codes) and not flag:
			end = i + 2
			flag = True
		if (end > start) and (start != -1):
			result.add((start + offset, end + offset))
		i = i + 3
	return result


def get_gc(string):
	gc = ((string.count('G') + string.count('C')) / len(string)) * 100
	return gc


def get_info(string, pos):
	length = pos[1] - pos[0] + 1
	gc = get_gc(string[pos[0]:pos[1]+1])
	return str(pos[0]), str(pos[1]), str(length), str(gc)


def orf(seq):
	result_info = []
	strings = [seq, seq[1:], seq[2:]]
	for index, string in enumerate(strings):
		# print(index)
		# print(string)
		positions = read_by_three(string, index)
		positions = sorted(positions, key=operator.itemgetter(0))
		# print(positions)
		for pos in positions:
			result_info.append(get_info(seq, pos))
	# print(result_info)
	# print(len(result_info))
	return result_info


def run(finput, foutput, label_dataset):
	file = open(foutput, 'a')
	file.write('nameseq,maximum_ORF_length,minimum_ORF_length,std_ORF_length,average_ORF_length,cv_ORF_length,' + ''
			   + 'maximum_GC_content_ORF,minimum_GC_content_ORF,std_GC_content_ORF,' + ''
			   + 'average_GC_content_ORF,cv_GC_content_ORF,label')
	file.write('\n')
	for seq_record in SeqIO.parse(finput, 'fasta'):
		seq = seq_record.seq
		seq = seq.upper()
		name_seq = seq_record.name
		file.write('%s,' % name_seq)
		measures = orf(seq)
		if len(measures) > 0:
			length_orf = []
			gc_mea = []
			for values in measures:
				length_orf.append(int(values[2]))
				gc_mea.append(float(values[3]))
			# print(length_orf)
			# print(gc_mea)
			file.write('%s,' % max(length_orf))
			file.write('%s,' % min(length_orf))
			file.write('%s,' % np.std(length_orf))
			file.write('%s,' % np.mean(length_orf))
			file.write('%s,' % scipy.stats.variation(length_orf))
			file.write('%s,' % max(gc_mea))
			file.write('%s,' % min(gc_mea))
			file.write('%s,' % np.std(gc_mea))
			file.write('%s,' % np.mean(gc_mea))
			file.write('%s,' % scipy.stats.variation(gc_mea))
			file.write('%s' % label_dataset)
			file.write('\n')
		else:
			file.write('0,0,0,0,0,0,0,0,0,0,')
			file.write('%s' % label_dataset)
			file.write('\n')
		print('Recorded Sequence: %s' % name_seq)
	return



def CodingClass(finput, foutput, label_dataset):
	start_time = time.time()
	run(finput, foutput, label_dataset)
	print('Computation time %s senconds' % (time.time() - start_time))


In [41]:
def look_up_position_prob(value, base, position_para, position_prob, position_weight):

	"""look up positional probability by base and value"""

	if float(value) < 0:
		return None
	for idx, val in enumerate(position_para):
		if float(value) >= val:
			return float(position_prob[base][idx]) * float(position_weight[base])


def look_up_content_prob(value, base, content_para, content_prob, content_weight):

	"""look up content probability by base and value"""

	if float(value) < 0:
		return None
	for idx, val in enumerate(content_para):
		if float(value) >= val:
			return float(content_prob[base][idx]) * float(content_weight[base])


def fickett_value_orf(seq, type_seq):

	"""calculate Fickett value. Input is DNA sequence"""

	position_prob = {
		'A': [0.94, 0.68, 0.84, 0.93, 0.58, 0.68, 0.45, 0.34, 0.20, 0.22],
		'C': [0.80, 0.70, 0.70, 0.81, 0.66, 0.48, 0.51, 0.33, 0.30, 0.23],
		'G': [0.90, 0.88, 0.74, 0.64, 0.53, 0.48, 0.27, 0.16, 0.08, 0.08],
		'T': [0.97, 0.97, 0.91, 0.68, 0.69, 0.44, 0.54, 0.20, 0.09, 0.09]}

	position_weight = {'A': 0.26, 'C': 0.18, 'G': 0.31, 'T': 0.33}
	position_para = [1.9, 1.8, 1.7, 1.6, 1.5, 1.4, 1.3, 1.2, 1.1, 0.0]

	content_prob = {
		'A': [0.28, 0.49, 0.44, 0.55, 0.62, 0.49, 0.67, 0.65, 0.81, 0.21],
		'C': [0.82, 0.64, 0.51, 0.64, 0.59, 0.59, 0.43, 0.44, 0.39, 0.31],
		'G': [0.40, 0.54, 0.47, 0.64, 0.64, 0.73, 0.41, 0.41, 0.33, 0.29],
		'T': [0.28, 0.24, 0.39, 0.40, 0.55, 0.75, 0.56, 0.69, 0.51, 0.58]}

	content_weight = {'A': 0.11, 'C': 0.12, 'G': 0.15, 'T': 0.14}
	content_para = [0.33, 0.31, 0.29, 0.27, 0.25, 0.23, 0.21, 0.17, 0]

	if len(seq) < 2:
		return 0
	fickett_score = 0
	seq = seq.upper()
	total_base = len(seq)
	A_content = float(seq.count('A')) / total_base
	C_content = float(seq.count('C')) / total_base
	G_content = float(seq.count('G')) / total_base
	if type_seq == 1:
		T_content = float(seq.count('T')) / total_base
	else:
		T_content = float(seq.count('U')) / total_base

	phase_0 = [seq[i] for i in range(0, len(seq)) if i % 3 == 0]
	phase_1 = [seq[i] for i in range(0, len(seq)) if i % 3 == 1]
	phase_2 = [seq[i] for i in range(0, len(seq)) if i % 3 == 2]
	
	A_position = max(phase_0.count('A'), phase_1.count('A'), phase_2.count('A')) / (min(phase_0.count('A'), phase_1.count('A'), phase_2.count('A')) + 1.0)

	C_position = max(phase_0.count('C'), phase_1.count('C'), phase_2.count('C')) / (min(phase_0.count('C'), phase_1.count('C'), phase_2.count('C')) + 1.0)

	G_position = max(phase_0.count('G'), phase_1.count('G'), phase_2.count('G')) / (min(phase_0.count('G'), phase_1.count('G'), phase_2.count('G')) + 1.0)

	if type_seq == 1:
		T_position = max(phase_0.count('T'), phase_1.count('T'), phase_2.count('T')) / (min(phase_0.count('T'), phase_1.count('T'), phase_2.count('T')) + 1.0)
	else:
		T_position = max(phase_0.count('U'), phase_1.count('U'), phase_2.count('U')) / (min(phase_0.count('U'), phase_1.count('U'), phase_2.count('U')) + 1.0)

	fickett_score += look_up_content_prob(A_content, 'A', content_para, content_prob, content_weight)
	fickett_score += look_up_content_prob(C_content, 'C', content_para, content_prob, content_weight)
	fickett_score += look_up_content_prob(G_content, 'G', content_para, content_prob, content_weight)
	fickett_score += look_up_content_prob(T_content, 'T', content_para, content_prob, content_weight)
	
	fickett_score += look_up_position_prob(A_position, 'A', position_para, position_prob, position_weight)
	fickett_score += look_up_position_prob(C_position, 'C', position_para, position_prob, position_weight)
	fickett_score += look_up_position_prob(G_position, 'G', position_para, position_prob, position_weight)
	fickett_score += look_up_position_prob(T_position, 'T', position_para, position_prob, position_weight)
	
	return fickett_score


def fickett_value_full_sequence(seq, type_seq):

	"""calculate Fickett from full sequence - CPC2"""

	position_para = [1.9, 1.8, 1.7, 1.6, 1.5, 1.4, 1.3, 1.2, 1.1, 0.0]
	content_para = [0.33, 0.31, 0.29, 0.27, 0.25, 0.23, 0.21, 0.19, 0.17, 0]

	position_prob = {
		'A': [0.51, 0.55, 0.57, 0.52, 0.48, 0.58, 0.57, 0.54, 0.50, 0.36],
		'C': [0.29, 0.44, 0.55, 0.49, 0.52, 0.60, 0.60, 0.56, 0.51, 0.38],
		'G': [0.62, 0.67, 0.74, 0.65, 0.61, 0.62, 0.52, 0.41, 0.31, 0.17],
		'T': [0.51, 0.60, 0.69, 0.64, 0.62, 0.67, 0.58, 0.48, 0.39, 0.24]}
	
	position_weight = {'A': 0.062, 'C': 0.093, 'G': 0.205, 'T': 0.154}
	content_weight = {'A': 0.084, 'C': 0.076, 'G': 0.081, 'T': 0.055}

	content_prob = {
		'A': [0.40, 0.55, 0.58, 0.58, 0.52, 0.48, 0.45, 0.45, 0.38, 0.19],
		'C': [0.50, 0.63, 0.59, 0.50, 0.46, 0.45, 0.47, 0.56, 0.59, 0.33],
		'G': [0.21, 0.40, 0.47, 0.50, 0.52, 0.56, 0.57, 0.52, 0.44, 0.23],
		'T': [0.30, 0.49, 0.56, 0.53, 0.48, 0.48, 0.52, 0.57, 0.60, 0.51]}

	if len(seq) < 2:
		return 0

	fickett_score = 0
	seq = seq.upper()
	total_base = len(seq)

	phase_0 = seq[::3]
	phase_1 = seq[1::3]
	phase_2 = seq[2::3]

	phase_0_A = phase_0.count('A')
	phase_1_A = phase_1.count('A')
	phase_2_A = phase_2.count('A')
	phase_0_C = phase_0.count('C')
	phase_1_C = phase_1.count('C')
	phase_2_C = phase_2.count('C')
	phase_0_G = phase_0.count('G')
	phase_1_G = phase_1.count('G')
	phase_2_G = phase_2.count('G')
	if type_seq == 1:
		phase_0_T = phase_0.count('T')
		phase_1_T = phase_1.count('T')
		phase_2_T = phase_2.count('T')
	else:
		phase_0_T = phase_0.count('U')
		phase_1_T = phase_1.count('U')
		phase_2_T = phase_2.count('U')

	A_content = float(phase_0_A + phase_1_A + phase_2_A) / total_base
	C_content = float(phase_0_C + phase_1_C + phase_2_C) / total_base
	G_content = float(phase_0_G + phase_1_G + phase_2_G) / total_base
	T_content = float(phase_0_T + phase_1_T + phase_2_T) / total_base
	A_position = max([phase_0_A, phase_1_A, phase_2_A]) / (min([phase_0_A, phase_1_A, phase_2_A]) + 1.0)
	C_position = max([phase_0_C, phase_1_C, phase_2_C]) / (min([phase_0_C, phase_1_C, phase_2_C]) + 1.0)
	G_position = max([phase_0_G, phase_1_G, phase_2_G]) / (min([phase_0_G, phase_1_G, phase_2_G]) + 1.0)
	T_position = max([phase_0_T, phase_1_T, phase_2_T]) / (min([phase_0_T, phase_1_T, phase_2_T]) + 1.0)

	fickett_score += look_up_content_prob(A_content, 'A', content_para, content_prob, content_weight)
	fickett_score += look_up_content_prob(C_content, 'C', content_para, content_prob, content_weight)
	fickett_score += look_up_content_prob(G_content, 'G', content_para, content_prob, content_weight)
	fickett_score += look_up_content_prob(T_content, 'T', content_para, content_prob, content_weight)

	fickett_score += look_up_position_prob(A_position, 'A', position_para, position_prob, position_weight)
	fickett_score += look_up_position_prob(C_position, 'C', position_para, position_prob, position_weight)
	fickett_score += look_up_position_prob(G_position, 'G', position_para, position_prob, position_weight)
	fickett_score += look_up_position_prob(T_position, 'T', position_para, position_prob, position_weight)

	return fickett_score


def header_fickett_score(foutput):
	file = open(foutput, 'a')
	file.write('%s,' % 'nameseq')
	file.write('%s,' % 'fickett_score-ORF')
	file.write('%s,' % 'fickett_score-full-sequence')
	file.write('label')
	file.write('\n')
	return


def calculate_sequences(finput, foutput, label_dataset, type_seq):
	header_fickett_score(foutput)
	for seq_record in SeqIO.parse(finput, 'fasta'):
		seq = seq_record.seq
		seq = seq.upper()
		name_seq = seq_record.name
		measure_orf = fickett_value_orf(seq, type_seq)
		measure_full = fickett_value_full_sequence(seq, type_seq)
		file = open(foutput, 'a')
		file.write('%s,' % name_seq)
		file.write('%s,' % str(measure_orf))
		file.write('%s,' % str(measure_full))
		file.write('%s' % str(label_dataset))
		file.write('\n')
		print('Recorded Sequence: %s' % name_seq)
	return


def FickettScore(finput, foutput, label_dataset):
	type_seq = 'DNA'
	calculate_sequences(finput, foutput, label_dataset, type_seq)

In [48]:
ExtractionTechniques("res.fasta", 'res_1.csv', 'dnc')
ExtractionTechniques("res.fasta", 'res_1.csv', 'tnc')
FourierClass("res.fasta", 'res_1.csv', 'mRNA')
EntropyClass("res.fasta", 'res_1.csv')
AccumulatedNucleotideFrequency("res.fasta", 'res_1.csv', '')
CodingClass("res.fasta", 'res_1.csv', 'DNA')
FickettScore("res.fasta", 'res_1.csv', 'DNA')

Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Sequence Analyzed!!
Sequence Analyzed!!
Sequence Analyzed!!
Sequence Analyzed!!
Sequence Analyzed!!
Recorded Sequence!!!
Recorded Sequence!!!
Recorded Sequence!!!
Recorded Sequence!!!
Recorded Sequence!!!
Dataset res.fasta: Found File
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Computation time 0.06989479064941406 senconds
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3
Recorded Sequence: BA000007.3